# CNN Implementation Tool
### Pat Dayton | 12 November 2018 | CS6301 | Dr. Redfern

In [1]:
!pip3 install keras
!pip3 install tqdm
!pip install tqdm
# !conda install keras
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras

from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet import MobileNet
from keras.applications.inception_v3 import InceptionV3

from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
# Input Shape
rows = 224
cols = 224
chan = 3

In [4]:
model_vgg19        = VGG19(include_top = False, input_shape=(rows, cols, chan))
model_resnet50     = ResNet50(include_top = False, input_shape=(rows, cols, chan))
model_mobilenet    = MobileNet(weights = None, include_top = False, input_shape=(rows, cols, chan))
model_inception_v3 = InceptionV3(include_top = False, input_shape=(rows, cols, chan))

/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [5]:
# Hardware Model
memory_bandwidth  = 10e9 # B/sec (GB)
internal_mem_size = 16e6 # Bytes (MB)
mnk = (32, 32, 32)       # mat_mul primitive size in BLAS notation
parallel_matmuls = 1     # number of matrix multiplication primitives operating in parallel
matmul_rate = 1e9 / 32   # number of matrix multiplication primitive completions per second
vec_prim_size = 32       # vector primitive size Nx1
parallel_vec_prims = 1   # number of vector primitives operating in parallel
vec_prim_rate = 1e9      # number of vector primitive completions per second

In [50]:
# Model Choice - uncomment the model/modelname you want to use

# model = model_mobilenet
# modelname = 'mobilenet_v1'

model = model_vgg19
modelname = 'vgg19'

# model = model_resnet50
# modelname = 'resnet50'

# model = model_inception_v3
# modelname = 'inception_v3'

In [51]:
layer_names = []
layer_types = []

Ni = []
Lr = []
Lc = []

No = []
Mr = []
Mc = []

Fr = []
Fc = []
Sr = []
Sc = []

f_mem = []

# first we loop through the model and grab information
# from model

for layer in model.layers:
    # create a list of layer types
    layer_type = layer.__class__.__name__
    layer_types.append(layer_type)
    
    # create a list of layer names
    layer_names.append(layer.name)
    
    # create lists for Ni, Lr, Lc, No, Mr, Mc
    if layer.__class__.__name__ in ['Concatenate', 'Add']:
        # these types of layers take in multiple shapes. For now
        # we can just model as the output shape
        Lr.append(layer.output_shape[1]) # rows
        Lc.append(layer.output_shape[2]) # columns
        Ni.append(layer.output_shape[3]) # channels
        Mr.append(layer.output_shape[1]) # rows
        Mc.append(layer.output_shape[2]) # columns
        No.append(layer.output_shape[3]) # channels
    else:
        # create a list of layer inputs Ni, Lr, Lc
        if (len(layer.input_shape) == 2):
            Lr.append(layer.input_shape[1]) # rows
            Lc.append(1)
            Ni.append(1)
        elif (len(layer.input_shape) == 3):
            Lr.append(layer.input_shape[1]) # rows
            Lc.append(layer.input_shape[2]) # columns
            Ni.append(1)
        else:
            Lr.append(layer.input_shape[1]) # rows
            Lc.append(layer.input_shape[2]) # columns
            Ni.append(layer.input_shape[3]) # channels

        # create lists for layer outputs No, Mr, Mc
        if (len(layer.output_shape) == 2):
            Mr.append(layer.output_shape[1]) # rows
            Mc.append(1)
            No.append(1)
        elif (len(layer.output_shape) == 3):
            Mr.append(layer.output_shape[1]) # rows
            Mc.append(layer.output_shape[2]) # columns
            No.append(1)
        else:
            Mr.append(layer.output_shape[1]) # rows
            Mc.append(layer.output_shape[2]) # columns
            No.append(layer.output_shape[3]) # channels
    
    
    # create lists for filters: Fr, Fc, Sr, Sc
    if 'filters' in layer.get_config():
        Fr.append(layer.kernel_size[0])
        Fc.append(layer.kernel_size[1])
        Sr.append(layer.strides[0])
        Sc.append(layer.strides[1])
    else:
        Fr.append(1)
        Fc.append(1)
        Sr.append(1)
        Sc.append(1)
        
    # determine filter memory
    f_mem.append(layer.count_params())
    

In [52]:
in_mem = []
in_move = []
in_us = []
out_mem = []
out_move = []
f_move = []
f_us = []
f_group = []
bandwidth = []
in_move_time = []
out_move_time = []
out_ds = []
f_move_time = []
tot_move_time = []
M = []
N = []
K = []
MACS = []
MACS_per_cycle = []
time_ser = []
time_par = []
matrix_compute = []
vector_compute = []
total_compute = []

for i, layer in enumerate(layer_names):

    # append memory of input tensor to in_mem
    in_mem.append(Ni[i]*Lr[i]*Lc[i])
    
    # see if input tensor fits on internal memory
    # first layer also always starts from off device
    if in_mem[i] > internal_mem_size or i == 0:
        in_move.append(1)
    else: in_move.append(0)
        
    # in upsampling will be not be included in any of these models
    in_us.append(1)
    
    # append memory of output tensor to out_mem
    out_mem.append(No[i]*Mr[i]*Mc[i])
    
    # see if output tensor fits on internal memory
    if out_mem[i] > internal_mem_size:
        out_move.append(1)
    else: out_move.append(0)
       
    # filter coefficients will always be off device
    f_move.append(1) # assume filter coefficients will always need be moved
    
    # filter grouping
    f_group.append(Ni[i])
    
    # filter upsampling
    f_us.append(1) # no upsampling for given models
    
    # determine output feature map downsampling ratio
    # We can assume it's simply the stride rate, 
    # and that the stride rate will be the same for row and column
    out_ds.append(Sr[i]) 
    
    # determine bandwidth
    bandwidth.append((in_mem[i]*in_move[i]) + (out_mem[i]*out_move[i]) + (f_mem[i]*f_move[i]))
    
    # in, out, filter movement times
    in_move_time.append((in_mem[i]*in_move[i]) / memory_bandwidth)
    out_move_time.append((out_mem[i]*out_move[i]) / memory_bandwidth)
    f_move_time.append((f_mem[i]*f_move[i]) / memory_bandwidth)
    
    # total movement time
    tot_move_time.append(bandwidth[i] / memory_bandwidth)
    
    # M, N, K
    M.append(No[i])
    N.append(Lr[i] * Lc[i])
    K.append(Ni[i]*Fr[i]*Fc[i])
    
    # MACs
    MACS.append(Ni[i] * No[i] * Fr[i] * Fc[i] * Mr[i] * Mc[i])
    
    # Compute Times
    
    # CNN Style Convolution Compute Time
    # based on hardware matrix compute parameters
    if layer_types[i] in ['Conv2D','DepthwiseConv2D']:
        MACS_per_cycle = mnk[0]*mnk[1]*mnk[2] # matmul rate has a /32 built in so we need all three
        matrix_compute.append(((MACS[i] / MACS_per_cycle) / matmul_rate) / parallel_matmuls)
        vector_compute.append(0)
        total_compute.append(matrix_compute[i])
    elif layer_types[i] in ['ReLU', 'Add']: 
        # Bias and RELU are free
        matrix_compute.append(0)
        vector_compute.append(0)
        total_compute.append(0)
    else: #Pooling included here. I left Activations other than ReLU to this bin as well
        matrix_compute.append(0)
        vector_compute.append(((max(out_mem[i], in_mem[i]) / vec_prim_size) / vec_prim_rate) / parallel_vec_prims )
        total_compute.append(vector_compute[i])
        
        
    # Time Serial
    time_ser.append(tot_move_time[i] + total_compute[i])
    
    # Time Parallel
    time_par.append(max(tot_move_time[i], total_compute[i]))

In [53]:
# Create a summation row for the csv
in_move_time_sum = sum(in_move_time)
f_move_time_sum = sum(f_move_time)
out_move_time_sum = sum(out_move_time)
tot_move_time_sum = sum(tot_move_time)
matrix_compute_sum = sum(matrix_compute)
vector_compute_sum = sum(vector_compute)
total_compute_sum = sum(total_compute)
time_ser_sum = sum(time_ser)
time_par_sum = sum(time_par)

In [54]:
data_tuples = list(zip(
    layer_names,
    layer_types, 
    Ni, 
    Lr, 
    Lc,
    in_mem,
#     in_move,
    in_us,
    No, 
    Mr, 
    Mc,
    out_mem,
#     out_move,
    out_ds,
    Fr, 
    Fc, 
    Sr, 
    Sc,
    f_us,
    f_group,
    f_mem,
#     f_move,
    bandwidth,
    in_move_time,
    out_move_time,
    f_move_time,
    tot_move_time,
#     M, 
#     N,
#     K,
#     MACS,
    time_ser,
    time_par,
    matrix_compute,
    vector_compute,
    total_compute))

df_cols = [
    'Layer Names',
    'Layer Types', 
    'Ni', 
    'Lr', 
    'Lc',
    'Input Mem',
#     'Input Move?',
    'Input Upsampling Ratio',
    'No', 
    'Mr', 
    'Mc',
    'Output Mem',
#     'Output Move?',
    'Output Downsampling Ratio',
    'Fr', 
    'Fc', 
    'Sr', 
    'Sc',
    'Filter Upsampling Ratio',
    'Filter Grouping', 
    'Filter Mem',
#     'Filter Move?',
    'Bandwidth',
    'Input Move Time',
    'Output Move Time',
    'Filter Move Time',
    'Total Move Time',
#     'M', 
#     'N',
#     'K',
#     'MACS',
    'Serial Time',
    'Parallel Time',
    'Matrix Compute Time',
    'Vector Compute Time',
    'Total Compute Time']

df1 = pd.DataFrame(data_tuples, columns=df_cols)

df1.head()

,Layer Names,Layer Types,Ni,Lr,Lc,Input Mem,Input Upsampling Ratio,No,Mr,Mc,Output Mem,Output Downsampling Ratio,Fr,Fc,Sr,Sc,Filter Upsampling Ratio,Filter Grouping,Filter Mem,Bandwidth,Input Move Time,Output Move Time,Filter Move Time,Total Move Time,Serial Time,Parallel Time,Matrix Compute Time,Vector Compute Time,Total Compute Time
0,input_1,InputLayer,3,224,224,150528,1,3,224,224,150528,1,1,1,1,1,1,3,0,150528,0.000015,0.0,0.000000e+00,1.505280e-05,0.000020,0.000015,0.000000,0.000005,0.000005
1,block1_conv1,Conv2D,3,224,224,150528,1,64,224,224,3211264,1,3,3,1,1,1,3,1792,1792,0.000000,0.0,1.792000e-07,1.792000e-07,0.000085,0.000085,0.000085,0.000000,0.000085
2,block1_conv2,Conv2D,64,224,224,3211264,1,64,224,224,3211264,1,3,3,1,1,1,64,36928,36928,0.000000,0.0,3.692800e-06,3.692800e-06,0.001810,0.001806,0.001806,0.000000,0.001806
3,block1_pool,MaxPooling2D,64,224,224,3211264,1,64,112,112,802816,1,1,1,1,1,1,64,0,0,0.000000,0.0,0.000000e+00,0.000000e+00,0.000100,0.000100,0.000000,0.000100,0.000100
4,block2_conv1,Conv2D,64,112,112,802816,1,128,112,112,1605632,1,3,3,1,1,1,64,73856,73856,0.000000,0.0,7.385600e-06,7.385600e-06,0.000911,0.000903,0.000903,0.000000,0.000903


In [55]:
# Create dataframe for summation layer
data_tuples = [[in_move_time_sum, f_move_time_sum, out_move_time_sum, tot_move_time_sum,
        matrix_compute_sum, vector_compute_sum, total_compute_sum, time_ser_sum, time_par_sum]]

df2 = pd.DataFrame(data_tuples, columns = ('Input Move Time',  'Filter Move Time', 
                                         'Output Move Time', 'Total Move Time', 
                                         'Matrix Compute Time',  'Vector Compute Time',
                                         'Total Compute Time', 'Serial Time',  'Parallel Time'))
df2.head()

,Input Move Time,Filter Move Time,Output Move Time,Total Move Time,Matrix Compute Time,Vector Compute Time,Total Compute Time,Serial Time,Parallel Time
0,0.000015,0.002002,0.0,0.002017,0.019051,0.000196,0.019247,0.021265,0.019258


In [56]:
df3 = pd.concat([df1,df2], keys=['layers', 'summations'], sort=False)
# df3 = df1.append(df2, sort = False)
df3.to_csv('{0}_{1}x{2}_output.csv'.format(modelname, rows, cols))
df3

Layer Names   Layer Types     Ni     Lr     Lc  Input Mem  \
layers     0        input_1    InputLayer    3.0  224.0  224.0   150528.0   
           1   block1_conv1        Conv2D    3.0  224.0  224.0   150528.0   
           2   block1_conv2        Conv2D   64.0  224.0  224.0  3211264.0   
           3    block1_pool  MaxPooling2D   64.0  224.0  224.0  3211264.0   
           4   block2_conv1        Conv2D   64.0  112.0  112.0   802816.0   
           5   block2_conv2        Conv2D  128.0  112.0  112.0  1605632.0   
           6    block2_pool  MaxPooling2D  128.0  112.0  112.0  1605632.0   
           7   block3_conv1        Conv2D  128.0   56.0   56.0   401408.0   
           8   block3_conv2        Conv2D  256.0   56.0   56.0   802816.0   
           9   block3_conv3        Conv2D  256.0   56.0   56.0   802816.0   
           10  block3_conv4        Conv2D  256.0   56.0   56.0   802816.0   
           11   block3_pool  MaxPooling2D  256.0   56.0   56.0   802816.0   
           12  block4_conv1        Conv2D  256.0   28.0   28.0   200704.0   
           13  block4_conv2        Conv2D  512.0   28.0   28.0   401408.0   
           14  block4_conv3        Conv2D  512.0   28.0   28.0   401408.0   
           15  block4_conv4        Conv2D  512.0   28.0   28.0   401408.0   
           16   block4_pool  MaxPooling2D  512.0   28.0   28.0   401408.0   
           17  block5_conv1        Conv2D  512.0   14.0   14.0   100352.0   
           18  block5_conv2        Conv2D  512.0   14.0   14.0   100352.0   
           19  block5_conv3        Conv2D  512.0   14.0   14.0   100352.0   
           20  block5_conv4        Conv2D  512.0   14.0   14.0   100352.0   
           21   block5_pool  MaxPooling2D  512.0   14.0   14.0   100352.0   
summations 0            NaN           NaN    NaN    NaN    NaN        NaN   

               Input Upsampling Ratio     No     Mr     Mc  Output Mem  \
layers     0                      1.0    3.0  224.0  224.0    150528.0   
           1                      1.0   64.0  224.0  224.0   3211264.0   
           2                      1.0   64.0  224.0  224.0   3211264.0   
           3                      1.0   64.0  112.0  112.0    802816.0   
           4                      1.0  128.0  112.0  112.0   1605632.0   
           5                      1.0  128.0  112.0  112.0   1605632.0   
           6                      1.0  128.0   56.0   56.0    401408.0   
           7                      1.0  256.0   56.0   56.0    802816.0   
           8                      1.0  256.0   56.0   56.0    802816.0   
           9                      1.0  256.0   56.0   56.0    802816.0   
           10                     1.0  256.0   56.0   56.0    802816.0   
           11                     1.0  256.0   28.0   28.0    200704.0   
           12                     1.0  512.0   28.0   28.0    401408.0   
           13                     1.0  512.0   28.0   28.0    401408.0   
           14                     1.0  512.0   28.0   28.0    401408.0   
           15                     1.0  512.0   28.0   28.0    401408.0   
           16                     1.0  512.0   14.0   14.0    100352.0   
           17                     1.0  512.0   14.0   14.0    100352.0   
           18                     1.0  512.0   14.0   14.0    100352.0   
           19                     1.0  512.0   14.0   14.0    100352.0   
           20                     1.0  512.0   14.0   14.0    100352.0   
           21                     1.0  512.0    7.0    7.0     25088.0   
summations 0                      NaN    NaN    NaN    NaN         NaN   

               Output Downsampling Ratio   Fr   Fc   Sr   Sc  \
layers     0                         1.0  1.0  1.0  1.0  1.0   
           1                         1.0  3.0  3.0  1.0  1.0   
           2                         1.0  3.0  3.0  1.0  1.0   
           3                         1.0  1.0  1.0  1.0  1.0   
           4                         1.0  3.0  3.0  1.0  1.0   
      